In [ ]:
import re
import os
import tqdm
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint

from collections import defaultdict
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from collections import Counter
import gensim
import nltk
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from gensim.scripts.glove2word2vec import glove2word2vec

from joblib import dump
from joblib import load

In [ ]:
data_path = "data/html/*.html"

code2convos = dict()

pbar = tqdm.tqdm(sorted(list(glob(data_path))))
for path in pbar:
    # print(Path.cwd() / path)
    file_code = os.path.basename(path).split(".")[0]
    with open(path, "r", encoding="latin1") as fh:
            
        # get the file id to use it as key later on
        fid = os.path.basename(path).split(".")[0]

        # read the html file
        html_page = fh.read()

        # parse the html file with bs4 so we can extract needed stuff
        soup = BeautifulSoup(html_page, "html.parser")

        # grab the conversations with the data-testid pattern
        data_test_id_pattern = re.compile(r"conversation-turn-[0-9]+")
        conversations = soup.find_all("div", attrs={"data-testid": data_test_id_pattern})

        convo_texts = []

        for i, convo in enumerate(conversations):
            convo = convo.find_all("div", attrs={"data-message-author-role":re.compile( r"[user|assistant]") })
            if len(convo) > 0:
                role = convo[0].get("data-message-author-role")
                convo_texts.append({
                        "role" : role,
                        "text" : convo[0].text
                    }
                )
                
        code2convos[file_code] = convo_texts

In [ ]:
# let's see one of the conversations
pprint(code2convos["0031c86e-81f4-4eef-9e0e-28037abf9883"][0])

#### Things to do:
- Prompt matching with questions
- Feature Engineering
- Question Grades preparation
- Train/Test split
- Fitting a model for predicting the scores

#### Prompt Matching
> We want to match the prompts with the questions in the Homework Let's
> do it with a simple term frequency vectorizing method. For each prompt,
> we will come with a vector that represents it. We will do the same
> thing with each of the homework questions. Then, we will calculate the
> vectors distanance to do the matching

In [ ]:
prompts = []
code2prompts = defaultdict(list)
for code , convos in code2convos.items():
    user_prompts = []
    for conv in convos:
        if conv["role"] == "user":
            prompts.append(conv["text"])
            user_prompts.append(conv["text"])
    code2prompts[code] = user_prompts    

In [ ]:
prompts[0]

In [ ]:
questions = [
    """Initialize
*   First make a copy of the notebook given to you as a starter.
*   Make sure you choose Connect form upper right.
*   You may upload the data to the section on your left on Colab, than right click on the .csv file and get the path of the file by clicking on "Copy Path". You will be using it when loading the data.

""",
#####################
    """Load training dataset (5 pts)
    *  Read the .csv file with the pandas library
""",
#####################
"""Understanding the dataset & Preprocessing (15 pts)
Understanding the Dataset: (5 pts)
> - Find the shape of the dataset (number of samples & number of attributes). (Hint: You can use the **shape** function)
> - Display variable names (both dependent and independent).
> - Display the summary of the dataset. (Hint: You can use the **info** function)
> - Display the first 5 rows from training dataset. (Hint: You can use the **head** function)
Preprocessing: (10 pts)

> - Check if there are any missing values in the dataset. If there are, you can either drop these values or fill it with most common values in corresponding rows. **Be careful that you have enough data for training the  model.**

> - Encode categorical labels with the mappings given in the cell below. (Hint: You can use **map** function)
""",
#####################
"""Set X & y, split data (5 pts)

*   Shuffle the dataset.
*   Seperate your dependent variable X, and your independent variable y. The column health_metrics is y, the rest is X.
*   Split training and test sets as 80% and 20%, respectively.
""",
#####################
"""Features and Correlations (10 pts)

* Correlations of features with health (4 points)
Calculate the correlations for all features in dataset. Highlight any strong correlations with the target variable. Plot your results in a heatmap.

* Feature Selection (3 points)
Select a subset of features that are likely strong predictors, justifying your choices based on the computed correlations.

* Hypothetical Driver Features (3 points)
Propose two hypothetical features that could enhance the model's predictive accuracy for Y, explaining how they might be derived and their expected impact. Show the resulting correlations with target variable.

* __Note:__ You get can get help from GPT.
""",
#####################
"""Tune Hyperparameters (20 pts)
* Choose 2 hyperparameters to tune. You can use the Scikit learn decision tree documentation for the available hyperparameters *(Hyperparameters are listed under "Parameters" in the documentation)*. Use GridSearchCV for hyperparameter tuning, with a cross-validation value of 5. Use validation accuracy to pick the best hyper-parameter values. (15 pts)
-Explain the hyperparameters you chose to tune. *(What are the hyperparameters you chose? Why did you choose them?)* (5 pts)
""",
#####################
"""Re-train and plot the decision tree with the hyperparameters you have chosen (15 pts)
- Re-train model with the hyperparameters you have chosen in part 5). (10 pts)
- Plot the tree you have trained. (5 pts)
Hint: You can import the **plot_tree** function from the sklearn library.
""",
#####################
"""Test your classifier on the test set (20 pts)
- Predict the labels of testing data using the tree you have trained in step 6. (10 pts)
- Report the classification accuracy. (2 pts)
- Plot & investigate the confusion matrix. Fill the following blanks. (8 pts)
> The model most frequently mistakes class(es) _________ for class(es) _________.
Hint: You can use the confusion_matrix function from sklearn.metrics
""",
#####################
"""Find the information gain on the first split (10 pts)""",
#####################
]

In [ ]:
"""
glove_input_file = 'glove/glove.6B.100d.txt'
word2vec_output_file = 'glove/glove.6B.100d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

glove_input_file = 'glove/glove.6B.50d.txt'
word2vec_output_file = 'glove/glove.6B.50d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

glove_input_file = 'glove/glove.6B.200d.txt'
word2vec_output_file = 'glove/glove.6B.200d.word2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
"""

In [ ]:
# glove_model = load('joblib_models/glove_model_50d.joblib')
glove_model = KeyedVectors.load_word2vec_format('glove/glove.6B.50d.word2vec.txt', binary=False)
dump(glove_model, "joblib_models/glove_model_50d.joblib")
def preprocess_and_tokenize(text):
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalpha()]


In [ ]:
def vectorize_prompts(prompts, model):
    vectorized = []
    for prompt in prompts:
        words = preprocess_and_tokenize(prompt)
        word_vectors = [model[word] for word in words if word in model.key_to_index]

        if len(word_vectors) == 0:
            vectorized.append(np.zeros(model.vector_size))  
        else:
            vectorized.append(np.mean(word_vectors, axis=0)) 

    return pd.DataFrame(vectorized)

code2prompts_glove = dict()

for code, user_prompts in code2prompts.items():
    if len(user_prompts) > 0:
        vectorized_df = vectorize_prompts(user_prompts, glove_model)
        code2prompts_glove[code] = vectorized_df
    else:
        print(f"{code}.html has no prompts")
        
print(code2prompts_glove["089eb66d-4c3a-4f58-b98f-a3774a2efb34"].shape)

In [ ]:
code2prompts_glove["089eb66d-4c3a-4f58-b98f-a3774a2efb34"].head()

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(prompts + questions)
questions_TF_IDF = pd.DataFrame(vectorizer.transform(questions).toarray(), columns=vectorizer.get_feature_names_out())
questions_TF_IDF.head()

In [ ]:
code2prompts_tf_idf = dict()
for code, user_prompts in code2prompts.items():
    if len(user_prompts) == 0:
        print(code+".html")
        continue
    prompts_TF_IDF = pd.DataFrame(vectorizer.transform(user_prompts).toarray(), columns=vectorizer.get_feature_names_out())
    code2prompts_tf_idf[code] = prompts_TF_IDF

In [ ]:
code2prompts_tf_idf["089eb66d-4c3a-4f58-b98f-a3774a2efb34"].head()

In [ ]:
code2cosine = dict()
for code, user_prompts_tf_idf in code2prompts_tf_idf.items():
    code2cosine[code] = pd.DataFrame(cosine_similarity(questions_TF_IDF,user_prompts_tf_idf))


In [ ]:
code2cosine["089eb66d-4c3a-4f58-b98f-a3774a2efb34"]

In [ ]:
question_of_prompts = dict()

for code, df in code2cosine.items():
    max_indices = []

    for col in df.columns:
        max_index = df[col].idxmax()

        max_indices.append(max_index)

    question_of_prompts[code] = max_indices
    
question_of_prompts["089eb66d-4c3a-4f58-b98f-a3774a2efb34"]

In [ ]:
code2prompts_glove["089eb66d-4c3a-4f58-b98f-a3774a2efb34"].shape

In [ ]:
num_questions = len(questions)

new_code_counts = dict()

for code, indices in question_of_prompts.items():
    counts = Counter(indices)

    count_vector = [counts.get(i, 0) for i in range(num_questions)]

    new_code_counts[code] = count_vector

new_code_counts["04f91058-d0f8-4324-83b2-19c671f433dc"]

In [ ]:
vector_size = 50   

final_df = pd.DataFrame(index=code2prompts_glove.keys(), columns=[f'q{i}_feature_{j}' for i in range(num_questions) for j in range(vector_size)])

for code in code2prompts_glove.keys():
    glove_vectors = code2prompts_glove[code]
    question_indices = question_of_prompts[code]

    summed_vectors = np.zeros((num_questions, vector_size))
    prompt_counts = Counter(question_indices)

    for vector, question_idx in zip(glove_vectors, question_indices):
        scaled_vector = vector / prompt_counts[question_idx]
        summed_vectors[question_idx] += scaled_vector

    for i in range(num_questions):
        column_labels = [f'q{i}_feature_{j}' for j in range(vector_size)]
        final_df.loc[code, column_labels] = summed_vectors[i]

final_df.rename_axis('code', inplace=True)
final_df.head()

In [ ]:
scaler = StandardScaler()

normalized_data = scaler.fit_transform(final_df)

normalized_df = pd.DataFrame(normalized_data, index=final_df.index, columns=final_df.columns)

normalized_df.rename_axis('code', inplace=True)

# Feature Engineering
- Number of prompts that a user asked
- Number of complaints that a user makes e.g "the code gives this error!"
- User prompts average number of characters

In [122]:
coefficients = [0.1,0.05,0.15,0.05,0.1,0.2,0.15,0.1,0.1]

modified_code_counts = {}

for code, count_vector in new_code_counts.items():
    modified_vector = [count * coeff for count, coeff in zip(count_vector, coefficients)]
    modified_code_counts[code] = modified_vector

modified_counts_df = pd.DataFrame.from_dict(modified_code_counts, orient='index')

modified_counts_df.index.name = 'code'

extended_df = normalized_df.merge(modified_counts_df, left_on='code', right_index=True)

modified_count_col_names = [f'modified_count_{i}' for i in range(num_questions)]
extended_df.rename(columns=dict(zip(range(num_questions), modified_count_col_names)), inplace=True)

extended_df.shape

(123, 459)

In [ ]:
code2questionmapping = dict()
for code, cosine_scores in code2cosine.items():
    code2questionmapping[code] = code2cosine[code].max(axis=1).tolist()

question_mapping_scores = pd.DataFrame(code2questionmapping).T
question_mapping_scores.reset_index(inplace=True)
question_mapping_scores.rename(columns={i: f"Q_{i}" for i in range(len(questions))}, inplace=True)
question_mapping_scores.rename(columns={"index" : "code"}, inplace=True)

question_mapping_scores.head()

In [ ]:
merged_df = question_mapping_scores.merge(extended_df, on='code', how='left')
# merged_df = question_mapping_scores.merge(modified_counts_df, on='code', how='left')
merged_df.shape

In [ ]:
# reading the scores
scores = pd.read_csv("data/scores.csv", sep=",")
scores["code"] = scores["code"].apply(lambda x: x.strip())

# selecting the columns we need and we care
scores = scores[["code", "grade"]]

# show some examples
scores.head()

In [ ]:
# Let's check grades distribution

plt.title('Histogram Grades')
plt.hist(scores["grade"], rwidth=.8, bins=np.arange(min(scores["grade"]), max(scores["grade"])+2) - 0.5)
plt.ylabel('Count')
plt.show()

#### Merging scores with features

In [ ]:
from joblib import dump

temp_df = pd.merge(merged_df, scores, on='code', how="left")
temp_df.dropna(inplace=True)
temp_df.drop_duplicates("code",inplace=True, keep="first")

temp_df = temp_df[temp_df['grade'] >= 70]

temp_df['grade'] = temp_df['grade'] - 70

scaler = StandardScaler()

temp_df['grade'] = scaler.fit_transform(temp_df[['grade']])
temp_df.head()


In [ ]:
X = temp_df[temp_df.columns[1:-1]].to_numpy()
y = temp_df["grade"].to_numpy()
print(X.shape, y.shape)

#### Train/Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train set size:", len(X_train))
print("Test set size:", len(X_test))

#### Fitting a model

#### Predicting and Analyzing 

In [ ]:
from joblib import dump

regressor = DecisionTreeRegressor(random_state=0,criterion='squared_error', max_depth=5)
regressor.fit(X_train, y_train)


In [ ]:
extracted_MSEs = regressor.tree_.impurity   
for idx, MSE in enumerate(regressor.tree_.impurity):
    print("Node {} has MSE {}".format(idx,MSE))

In [ ]:
# Prediction
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)

# Calculation of Mean Squared Error (MSE)
print("MSE Train:", mean_squared_error(y_train,y_train_pred))
print("MSE TEST:", mean_squared_error(y_test,y_test_pred))

print("R2 Train:", r2_score(y_train,y_train_pred))
print("R2 TEST:", r2_score(y_test,y_test_pred))


In [ ]:
from sklearn.linear_model import Lasso

lasso_regressor = Lasso(alpha=0.01, random_state=0)  
lasso_regressor.fit(X_train, y_train)

y_train_pred_lasso = lasso_regressor.predict(X_train)
y_test_pred_lasso = lasso_regressor.predict(X_test)

print("\nLasso Regression:")
print("MSE Train:", mean_squared_error(y_train, y_train_pred_lasso))
print("MSE Test:", mean_squared_error(y_test, y_test_pred_lasso))
print("R2 Train:", r2_score(y_train, y_train_pred_lasso))
print("R2 Test:", r2_score(y_test, y_test_pred_lasso))


In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

pca = PCA(n_components=0.95)  # Keep 95% of variance
linear_reg = LinearRegression()
pcr = make_pipeline(pca, linear_reg)

pcr.fit(X_train, y_train)

y_train_pred_pcr = pcr.predict(X_train)
y_test_pred_pcr = pcr.predict(X_test)

print("\nPrincipal Component Regression:")
print("MSE Train:", mean_squared_error(y_train, y_train_pred_pcr))
print("MSE Test:", mean_squared_error(y_test, y_test_pred_pcr))
print("R2 Train:", r2_score(y_train, y_train_pred_pcr))
print("R2 Test:", r2_score(y_test, y_test_pred_pcr))


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

random_forest_regressor = RandomForestRegressor(random_state=42, n_estimators=40)  # n_estimators can be adjusted

random_forest_regressor.fit(X_train, y_train)

y_train_pred_rf = random_forest_regressor.predict(X_train)
y_test_pred_rf = random_forest_regressor.predict(X_test)

print("Random Forest Regressor:")
print("MSE Train:", mean_squared_error(y_train, y_train_pred_rf))
print("MSE Test:", mean_squared_error(y_test, y_test_pred_rf))

print("R2 Train:", r2_score(y_train, y_train_pred_rf))
print("R2 Test:", r2_score(y_test, y_test_pred_rf))


In [ ]:
"""
In this cell, almost all combinations have been tried.
PCA+RF, RF+Lasso, Lasso+RF, RF + Average, PCR + Average and etc.
"""

X_train_ensemble = np.column_stack((y_train_pred_rf, y_train_pred_lasso)) 
X_test_ensemble = np.column_stack((y_test_pred_rf, y_test_pred_lasso))

ensemble_regressor = DecisionTreeRegressor(random_state=42)
ensemble_regressor.fit(X_train_ensemble, y_train)

y_train_pred_ensemble = ensemble_regressor.predict(X_train_ensemble)
y_test_pred_ensemble = ensemble_regressor.predict(X_test_ensemble)

print("Ensemble Model with Decision Tree Regressor:")
print("MSE Train:", mean_squared_error(y_train, y_train_pred_ensemble))
print("MSE Test:", mean_squared_error(y_test, y_test_pred_ensemble))
print("R2 Train:", r2_score(y_train, y_train_pred_ensemble))
print("R2 Test:", r2_score(y_test, y_test_pred_ensemble))